## Question 6 (8 points)

Complete the function `order` so that given $x$ and $N$, it returns the histogram of samples from the order finding circuit.

The function `order` has

- Input: $x$ and $N$
- Returns: A histogram of samples

Notes:

- You are given a function named $U_x$ which implements $U_x |y\rangle \rightarrow |xy {\mod N}\rangle $ and returns its controlled version. Run the following cell to load the function. In order to use the function you should pass $x$ and $N$ as parameter.

    <pre>CU=Ux(x,N)</pre>

- You are also given the code for `qpe` and `iqft`.

In [1]:
import numpy as np
def Ux(x,N):

    k=1
    while(N>2**k):
        k=k+1
        
    u = np.zeros([2**k, 2**k], dtype = int) 

    for i in range(N):
        u[x*i%N][i]=1
    for i in range(N,2**k):
        u[i][i]=1
        

    XU = cirq.MatrixGate(u).controlled()
    return XU

In [2]:
import cirq
from cirq.circuits import InsertStrategy
from cirq import H, SWAP, CZPowGate
def iqft(n,qubits,circuit):
    
    #Swap the qubits
    for i in range(n//2):
        circuit.append(SWAP(qubits[i],qubits[n-i-1]), strategy = InsertStrategy.NEW)
     
    #For each qubit
    for i in range(n-1,-1,-1):
        #Apply CR_k gates where j is the control and i is the target
        k=n-i #We start with k=n-i
        for j in range(n-1,i,-1):
            #Define and apply CR_k gate
            crk = CZPowGate(exponent = -2/2**(k))
            circuit.append(crk(qubits[j],qubits[i]),strategy = InsertStrategy.NEW)
            k=k-1 #Decrement at each step
            
        #Apply Hadamard to the qubit
        circuit.append(H(qubits[i]),strategy = InsertStrategy.NEW)

def qpe(t,control, target, circuit, CU):
    
    #Apply Hadamard to control qubits
    circuit.append(cirq.H.on_each(control))
    
    #Apply CU gates
    for i in range(t):
        #Obtain the power of CU gate 
        CUi = CU**(2**i)
        #Apply CUi gate where t-i-1 is the control
        circuit.append(CUi(control[t-i-1],*target))
        
    #Apply inverse QFT
    iqft(t,control,circuit)

In [3]:
import cirq
from cirq import X, SWAP

def order(x,N):
    #Create a circuit
    circuit = cirq.Circuit()
    # YOUR CODE HERE

    # Number of control qubits needed (precision)
    t = int(np.ceil(np.log2(N)))  # so that 2^t ≥ N
    control = cirq.LineQubit.range(t)

    # Number of target qubits needed to represent N
    L = int(np.ceil(np.log2(N)))
    target = cirq.LineQubit.range(t, t + L)

    # Initialize target register to |1>
    circuit.append(cirq.X(target[0]))

    # Load controlled Ux gate
    CU = Ux(x, N)

    # Apply QPE 
    qpe(t, control, target, circuit, CU)


    #Measure the control register
    circuit.append(cirq.measure(*control, key='result'))

    #Sample the circuit
    s=cirq.Simulator()
    samples=s.run(circuit, repetitions=1000)

    # Return a histogram of samples
    results= samples.histogram(key='result')
    return results

In [4]:
#You can use this cell to test your function

results = order(4,31)
print(results)

Counter({0: 210, 13: 168, 19: 158, 6: 126, 26: 113, 25: 58, 7: 47, 12: 19, 20: 14, 24: 11, 27: 11, 5: 10, 18: 9, 14: 5, 15: 4, 28: 4, 8: 4, 29: 3, 16: 3, 30: 3, 11: 3, 4: 3, 2: 2, 10: 2, 31: 2, 22: 2, 9: 2, 23: 2, 3: 1, 21: 1})
